In [ ]:
import os
os.environ["BAYESMIX_EXE"] = "/home/m_gianella/Documents/GitHub/bayesmix/build/run_mcmc"

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from bayesmixpy import run_mcmc
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [ ]:
np.random.seed(123)

# Set true parameters
N = 500
Ncomp = 3
means = [-5.0, 0.0, 5.0]
sds = [0.5, 2.0, 0.25]
weights = np.ones(Ncomp)/Ncomp

cluster_allocs = tfd.Categorical(probs=weights).sample(N)
data = np.stack([tfd.Normal(means[cluster_allocs[i]], sds[cluster_allocs[i]]).sample() for i in range(N)])

In [ ]:
# Setup parameters for bayesmixpy
hier_params = \
"""
fixed_values {
  mean: 0.0
  var: 10.0
  shape: 2.0
  scale: 2.0
}
"""

mix_params = \
"""
dp_prior {
  totalmass: 1
}
num_components: 3
"""

algo_params = \
"""
algo_id: "BlockedGibbs"
rng_seed: 20201124
iterations: 2000
burnin: 1000
init_num_clusters: 3
"""

dens_grid = np.linspace(-7.5,7.5,1000)

In [ ]:
# Fit model using bayesmixpy
eval_dens, n_clus, clus_chain, best_clus = run_mcmc("NNxIG","TruncSB", data,
                                                    hier_params, mix_params, algo_params,
                                                    dens_grid, return_num_clusters=False,
                                                    return_clusters=False, return_best_clus=True)

In [ ]:
# Plot mean posterior density
plt.plot(dens_grid, np.exp(eval_dens.mean(axis=0)))
plt.hist(data, alpha=0.4, density=True)
for c in np.unique(best_clus):
    data_in_clus = data[best_clus == c]
    plt.scatter(data_in_clus, np.zeros_like(data_in_clus) + 0.01)
plt.title("Posterior estimated density")
plt.show()